In [1]:
!nvidia-smi

Sat Sep 25 07:05:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%time
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-1.9.0+cu102.html
!pip install -q torch-geometric
# !pip install -q torch-scatter
# !pip install -q torch-sparse 

     |████████████████████████████████| 8.0 MB 8.0 MB/s 
     |████████████████████████████████| 2.9 MB 6.4 MB/s 
     |████████████████████████████████| 308 kB 13.2 MB/s 
     |████████████████████████████████| 379 kB 44.2 MB/s 
     |████████████████████████████████| 45 kB 3.2 MB/s 
CPU times: user 169 ms, sys: 41.6 ms, total: 211 ms
Wall time: 16.3 s


In [3]:
# !pip show torch

In [73]:
import argparse
import os.path as osp
from tqdm import tqdm
from sklearn.cluster import KMeans

import torch
from torch.nn import ReLU
import torch.nn.functional as F

import torch_geometric.transforms as T
# from torch_geometric.datasets import OGB_MAG
from torch_geometric.datasets import DBLP
from torch_geometric.loader import NeighborLoader, HGTLoader
from torch_geometric.nn import Sequential, SAGEConv, Linear, to_hetero, HeteroConv

path = '../data/DBLP/'
dataset = DBLP(path)
data = dataset[0]

# We initialize conference node features with a single feature.
data['conference'].x = torch.ones(data['conference'].num_nodes, 1)

# train->test
train_input_nodes = ('author', data['author'].test_mask)
val_input_nodes = ('author', data['author'].val_mask)
kwargs = {'batch_size': 64, 'num_workers': 2, 'persistent_workers': True}

# بدون خوشه‌بندی
train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True,
                              input_nodes=train_input_nodes, **kwargs)

# Cluster input nodes
X = data.x_dict['author'].numpy()[data['author'].test_mask]
orig_index = data['author'].test_mask.nonzero(as_tuple=False).view(-1)
K = 10
k_means = KMeans(n_clusters=K, random_state=0).fit(X)
indices = []
for k in range(K):
    print(sum(k_means.labels_==k))
    indexes = [i for i,x in enumerate(k_means.labels_) if x == k]
    indices.extend(indexes)

train_clustered_nodes = [orig_index[x] for x in indices]
train_input_nodes_clustered = ('author', torch.LongTensor(train_clustered_nodes))
train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=False,
                              input_nodes=train_input_nodes_clustered, **kwargs)

val_loader = NeighborLoader(data, num_neighbors=[10] * 2,
                            input_nodes=val_input_nodes, **kwargs)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class HeteroGNN(torch.nn.Module):
    def __init__(self, metadata, hidden_channels, out_channels, num_layers):
        super().__init__()

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HeteroConv({
                edge_type: SAGEConv((-1, -1), hidden_channels)
                for edge_type in metadata[1]
            })
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)
            x_dict = {key: F.leaky_relu(x) for key, x in x_dict.items()}
        return self.lin(x_dict['author'])


model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=4,
                  num_layers=2)
model = model.to(device)

@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    batch = next(iter(train_loader))
    batch = batch.to(device)
    model(batch.x_dict, batch.edge_index_dict)


def train():
    model.train()
    i = 0
    total_examples = total_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        batch = batch.to(device)
        # if i<3:
        #   print(batch)
        # i += 1

        batch_size = batch['author'].batch_size
        out = model(batch.x_dict, batch.edge_index_dict)
        loss = F.cross_entropy(out[:batch_size], batch['author'].y[:batch_size])
        loss.backward()
        optimizer.step()

        total_examples += batch_size
        total_loss += float(loss) * batch_size

    return total_loss / total_examples


@torch.no_grad()
def test(loader):
    model.eval()

    total_examples = total_correct = 0
    for batch in tqdm(loader):
        batch = batch.to(device)
        batch_size = batch['author'].batch_size

        out = model(batch.x_dict, batch.edge_index_dict)#['author'][:batch_size]
        pred = out.argmax(dim=-1)

        total_examples += batch_size
        total_correct += int((pred[:batch_size] == batch['author'].y[:batch_size]).sum())

    return total_correct / total_examples


# init_params()  # Initialize parameters.
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# for epoch in range(1, 21):
#     loss = train()
#     val_acc = test(val_loader)
#     print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_acc:.4f}')

1133
19
337
537
85
127
217
342
361
99


In [74]:
# X = data.x_dict['author'].numpy()[data['author'].test_mask]
# print(X.shape)
# orig_index = data['author'].test_mask.nonzero(as_tuple=False).view(-1)
# len(index), 
# len(data['author'].test_mask.nonzero(as_tuple=False).view(-1))

In [75]:
# اجرا بدون خوشه‌بندی
train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=True,
                              input_nodes=train_input_nodes, **kwargs)
init_params()  # Initialize parameters.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 11):
    loss = train()
    val_acc = test(val_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_acc:.4f}')

100%|██████████| 7/7 [00:00<00:00, 25.77it/s]


Epoch: 01, Loss: 0.5648, Val: 0.8650


100%|██████████| 7/7 [00:00<00:00, 43.71it/s]


Epoch: 02, Loss: 0.1422, Val: 0.8600


100%|██████████| 7/7 [00:00<00:00, 52.31it/s]


...

Epoch: 08, Loss: 0.0058, Val: 0.8675


100%|██████████| 7/7 [00:00<00:00, 51.73it/s]


Epoch: 09, Loss: 0.0080, Val: 0.8750


100%|██████████| 7/7 [00:00<00:00, 51.12it/s]

Epoch: 10, Loss: 0.0033, Val: 0.8675


In [77]:
# اجرا با خوشه‌بندی
train_loader = NeighborLoader(data, num_neighbors=[10] * 2, shuffle=False,
                              input_nodes=train_input_nodes_clustered, **kwargs)
init_params()  # Initialize parameters.
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 11):
    loss = train()
    val_acc = test(val_loader)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_acc:.4f}')

100%|██████████| 7/7 [00:00<00:00, 47.64it/s]


Epoch: 01, Loss: 0.0096, Val: 0.8625


 69%|██████▊   | 35/51 [00:00<00:00, 39.26it/s]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4892b644d0>
Traceback (most recent call last):
 76%|███████▋  | 39/51 [00:00<00:00, 37.81it/s]  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f4892b644d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/da

Epoch: 02, Loss: 0.0501, Val: 0.8375


100%|██████████| 7/7 [00:00<00:00, 50.07it/s]


...

Epoch: 08, Loss: 0.0071, Val: 0.8475


100%|██████████| 7/7 [00:00<00:00, 52.64it/s]


Epoch: 09, Loss: 0.0034, Val: 0.8525


100%|██████████| 7/7 [00:00<00:00, 49.67it/s]

Epoch: 10, Loss: 0.0020, Val: 0.8525


In [65]:
train_input_nodes, train_input_nodes_clustered

(('author', tensor([ True,  True, False,  ...,  True,  True,  True])),
 ('author', tensor([   0,    4,    6,  ..., 2951, 3131, 3152])))

In [66]:
# pip show torch-geometric
orig_index

tensor([   0,    1,    3,  ..., 4054, 4055, 4056])

In [36]:
# dataset

In [38]:
# print(batch)

In [37]:
data

HeteroData(
  author={
    x=[4057, 334],
    y=[4057],
    train_mask=[4057],
    val_mask=[4057],
    test_mask=[4057]
  },
  paper={ x=[14328, 4231] },
  term={ x=[7723, 50] },
  conference={
    num_nodes=20,
    x=[20, 1]
  },
  (author, to, paper)={ edge_index=[2, 19645] },
  (paper, to, author)={ edge_index=[2, 19645] },
  (paper, to, term)={ edge_index=[2, 85810] },
  (paper, to, conference)={ edge_index=[2, 14328] },
  (term, to, paper)={ edge_index=[2, 85810] },
  (conference, to, paper)={ edge_index=[2, 14328] }
)

In [ ]:
# https://pytorch-geometric.readthedocs.io/en/latest/notes/heterogeneous.html
# https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/to_hetero_mag.py